In [ ]:
from pynq import Overlay

# Initialize overlay
ol = Overlay("./ethernet.bit")

# Reference DMA and channels
dma = ol.dma
dma_send = ol.dma.sendchannel
dma_recv = ol.dma.recvchannel

In [22]:
import socket
import threading
from pynq import allocate
import numpy as np

# Function to handle client connection
def handle_client(client_socket):
    try:
        while True:
            # Receive message from client
            message = client_socket.recv(1024)
            if not message:
                break
            
            # Transfer input buffer data to the DMA using Main Memory 2 Stream
            message_int = np.frombuffer(message, dtype=np.uint32)

            # Allocate DMA-compatible buffer and copy data into input buffer
            input_buffer = allocate(shape=message_int.shape, dtype=np.uint32)
            output_buffer = allocate(shape=message_int.shape, dtype=np.uint32)
            input_buffer[:] = message_int
            
            # Send and receive from the FIFO
            dma_send.transfer(input_buffer)
            dma_recv.transfer(output_buffer)
            
            # Echo the message back to the client
            client_socket.send(output_buffer.tobytes())
    except Exception as e:
        print(f"Error handling client: {e}")
    finally:
        # Close the connection
        client_socket.close()

# Function to start the server
def start_server(host, port):
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # Enable reuse of the address/port
    server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

    server.bind((host, port))
    server.listen(5)
    print(f"Server listening on {host}:{port}")
    
    while True:
        # Accept new connections
        client_socket, client_address = server.accept()
        print(f"Connection from {client_address}")
        
        # Start a new thread to handle the client
        client_thread = threading.Thread(target=handle_client, args=(client_socket,))
        client_thread.start()

In [ ]:
host = '192.168.2.99'
port = 5011

start_server(host, port)

Server listening on 192.168.2.99:5011
Connection from ('192.168.2.1', 65384)
Connection from ('192.168.2.1', 65400)
Connection from ('192.168.2.1', 65410)
Connection from ('192.168.2.1', 65422)
